In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(['NAME','EIN'], axis = 1)


In [3]:
# Replace values if count is less than 200
value_counts = application_df['APPLICATION_TYPE'].value_counts()
replace_values = value_counts[value_counts < 200].index # index retrieves the indices (unique values) from the value_counts
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(replace_values, 'Other')

In [4]:
# Look at CLASSIFICATION value counts for binning
class_count=application_df['CLASSIFICATION'].value_counts()
class_cutoff=class_count[class_count>1]
replace_value_class = class_count[class_count<1883].index
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(replace_value_class, 'Other')


In [5]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [6]:
# Split our preprocessed data into our features and target arrays
y= application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'], axis =1).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 20
hidden_nodes_layer2 = 15
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                880       
                                                                 
 dense_1 (Dense)             (None, 15)                315       
                                                                 
 dense_2 (Dense)             (None, 10)                160       
                                                                 
 dense_3 (Dense)             (None, 5)                 55        
                                                                 
 dense_4 (Dense)             (None, 1)                 6         
                                                                 
Total params: 1,416
Trainable params: 1,416
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 200)

Epoch 1/200
804/804 [==============================] - 8s 5ms/step - loss: 0.5843 - accuracy: 0.7126
Epoch 2/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5564 - accuracy: 0.7308
Epoch 3/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5523 - accuracy: 0.7316
Epoch 4/200
804/804 [==============================] - 5s 6ms/step - loss: 0.5510 - accuracy: 0.7325
Epoch 5/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5495 - accuracy: 0.7334
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7338
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7346
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7351
Epoch 9/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5467 - accuracy: 0.7349
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accura

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5620 - accuracy: 0.7254 - 761ms/epoch - 3ms/step
Loss: 0.5619821548461914, Accuracy: 0.7253644466400146


In [12]:
# Export our model to HDF5 file
from keras.models import save_model


model_filename ='Export/AlphabetSoupCharity_Optimization.h5'

# Save the trained model to an HDF5 file with additional options
save_model(nn, model_filename, save_format='h5')

print("Model saved to", model_filename)

Model saved to Export/AlphabetSoupCharity_Optimization.h5
